 # Предобработка данных

In [ ]:
import os
import re
import pandas as pd

Для подсчета слов использовалась команда ```wc -w```:

In [ ]:
# PATH -- путь к папке с исходными файлами

!find PATH -name '*.txt' -type f | xargs wc -w

Далее производилась обработка корпуса с помощью Ru Syntax (http://web-corpora.net/wsgi3/ru-syntax/), в результате были получены файл c разбором в формате ```.conll``` для каждого документа корпуса.

Из каждого полученного файла ```.conll``` были взяты лексемы и через ```_``` соединены с обозначением части речи (POS tagging). На выходе получились файлы с текстами каждого документа, в которых все слова стоят в начальной форме и рядом через ```_``` указана их часть речи.

In [ ]:
# PATH -- путь к папке с файлами .conll
# PATH2 -- путь для сохранения полученных файлов

for file in (os.listdir(PATH)):
    data = pd.read_csv(PATH + file, sep='\t',
                       names=['id', 'form', 'lemma', 'upos',
                              'xpos', 'feats', 'head', 'deprel',
                              'deps', 'misc'], low_memory=False)
    
    data['lemma_pos'] = data['lemma'] + '_' + data['upos']
    
    with open(PATH2 + file.replace('.conll', '_lemma_pos.txt'), 
              'w', encoding='utf-8') as fw:
        fw.write(' '.join(data['lemma_pos'].str.lower().astype('str')))

После этого в файлах все числа, знаки препинания, вхождения, которые определились программой RU Syntax как nonlex или как однобуквенные существительные, и сокращение _гг_, _гг._ от года были заменены на тег ```<w>```.

In [ ]:
# PATH -- путь к папке с файлами _lemma_pos.txt, полученным на предыдущем шаге
# PATH2 -- путь для сохранения файлов

pattern = re.compile('[^ ]*?(?:[«»()–,:]|_\W|___|sent|num|nonlex)[^ ]*?')
pattern2 = re.compile('\s(([a-zа-я]|гг)\.?_s)\s')
pattern3 = re.compile('<w> *?<w>')
pattern4 = re.compile('(<w> ?){2,}')

for file in os.listdir(PATH):
    with open(PATH + file, 'r', encoding='utf-8') as f:
        text = f.read()
        
        cleaned = re.sub(pattern, '<w>', text)
        cleaned2 = re.sub(pattern2, ' <w> ', cleaned)
        cleaned3 = re.sub(pattern3, '<w>', cleaned2)
        cleaned4 = re.sub(pattern4, '<w> ', cleaned3)
    
    with open(PATH2 + file, 'w', encoding='utf-8') as fw:
        fw.write(cleaned4)

Далее с помощью программы AntConc был составлен частотный список всех биграмм из текстов коллекции
и отсортирован по убыванию частотности. 

Из этого списка были удалены все вхождения, частотность которых == 1:

In [ ]:
# PATH -- путь к списку биграмм

data = pd.read_csv(PATH, names=['rank', 'freq', 'range', 'ngrams'],
                   sep='\t', low_memory=False).drop([0, 1], axis=0)
data = data.drop(data[data['freq'] == 1].index)

Также были удалены все вхождения, которые содержат тег ```<w>```, точку (инициалы или сокращения), показатель части речи без какого-либо слова или являются местоимением (кроме начинающихся на _с_, как возвратное _себя_, которое встречается в таких конструкциях, как _представлять себя_):

In [ ]:
pos = data['ngrams'].str.contains('[^с]_spro|apro|\W_[a-z]|<w>|\.', regex = True)
df = data.drop(pos[pos == True].index)

Затем были удалены все вхождения, которые не содержат показателя части речи:

In [ ]:
not_pos = df['ngrams'].str.contains('_', regex = True)
df = df.drop(not_pos[not_pos == False].index)

И наконец были удалены все биграммы, включающие слова из списка стоп-слов, который содержит преимущественно самые частотные союзы, местоимения, наречия и частицы.

In [ ]:
# PATH2 -- путь для сохранения файла

stopwords = open('stopwords.txt').readlines()
    
for word in stopwords:
    word = word.strip()
    
    with_stopwords1 = df['ngrams'].str.contains('^' + word, regex = True)
    df = df.drop(with_stopwords1[with_stopwords1 == True].index)
    
    with_stopwords2 = df['ngrams'].str.contains(' ' + word, regex = True)
    df = df.drop(with_stopwords2[with_stopwords2 == True].index)

df = df.sort_values(['range', 'freq'], ascending=False)
df.to_csv(PATH2, sep='\t', index=False, header=False)

В качестве лингвистического фильтра были выбраны 7 морфологических шаблонов: Verb + Noun, Noun + Verb, Prep + Noun, Noun + Prep, Verb + Prep, Adj + Noun, Adv + Verb. 
Для каждого из шаблонов были извлечены все соответствующие им выражения из списка биграмм, с условием, что они встречаются не менее, чем в 5 документах коллекции.

В случае выделения выражений по морфологическим шаблонам, в которых присутствует глагол (Verb + Noun, Verb + Prep, Adv + Verb), в них включались конструкции и с тегом ```_partcp``` (для других морфологических шаблонов код отличался только регулярным выражением):

In [ ]:
# PATH_bigrams -- путь к списку биграмм
# PATH2 -- путь для сохранения файла

data = pd.read_csv(PATH_bigrams, names=['id', 'freq', 'range', 'ngrams'],
                   sep='\t', low_memory=False)

data = data[data['range'] > 4]

prs = data['ngrams'].str.contains('\w+?_(?:v|partcp)\s\w+?_pr', regex = True)
df = data.drop(prs[prs == False].index)

df.to_csv(PATH2, sep='\t', index=False, header=False)

В результате было получено 7 файлов с биграммами, соответствующими морфологическим шаблонам, с частотностью для каждой биграммы и количеством документов, в которых она встречаются.

Для референтного корпуса были проделаны все те же шаги, что и для основного корпуса.